# GaP workflow

FILES WE NEED:
- input .d12 file (basis set, scf parameters)

INPUT FILES WE BUILD 
(PRIMITIVE CELL):
- OPTGEOM
- ELECTRONIC BAND STRUCTURE - DOS - EFFECTIVE MASS
- HIGH FREQ DIELECTRIC CONSTANT
- LOW FREQ DIELECTRIC CONSTANT
- PIZOELECTRIC CONSTANT
- ELASTIC CONSTANT

(SUPERCELL):
- 6X6X6 PHONONS 
- DISPERSION - DOS
- 4 0.1% DISPLACEMENTS 
- 


In [1]:
import copy
import os
from crystal_functions.file_readwrite import *
from pymatgen.symmetry.bandstructure import HighSymmKpath
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

from crystal_functions.convert import *


## Primitive cell

In [7]:
import sys 
sys.path.insert(-1,'../../../Imperial/crystal-code-tools/crystal_functions/crystal_functions/')

from file_readwrite import *
from convert import *

gap_input_initial = Crystal_input().from_file('./data/GaP_input.d12')
gap_gui_initial = Crystal_gui().read_cry_gui('./data/GaP_input.gui')
gap_pmg_initial = cry_gui2pmg(gap_gui_initial)
gap_pmg_intial_prim = SpacegroupAnalyzer(gap_pmg_initial).get_primitive_standard_structure(international_monoclinic=False)

slurm_file_crystal = './data/slurm_file_crystal.slurm'

file = open(slurm_file_crystal, 'r')
slurm_file_crystal_data = file.readlines()
file.close()

slurm_file_properties = './data/slurm_file_properties.slurm'

file = open(slurm_file_properties, 'r')
slurm_file_properties_data = file.readlines()
file.close()

functional_list = ['HSE06', 'PBE', 'PBE0', 'PBESOL', 'PBESOL0', 'PW91', 'M06', 'BLYP', 'B3LYP', 'B2GPPLYP']
functional_details = [['HSE06\n'],
                     ['EXCHANGE\n','PBE\n','CORRELAT\n','PBE\n'],
                     ['PBE0\n'],
                     ['EXCHANGE\n','PBESOL\n','CORRELAT\n','PBESOL\n'],
                     ['PBESOL0\n'],
                     [],
                     ['M06\n'],
                     ['EXCHANGE\n','BECKE\n','CORRELAT\n','LYP\n'],
                     ['B3LYP\n'],
                     ['B2GPPLYP\n']]

opt_list = ['OPTGEOM\n','FULLOPTG\n','FRACTION\n','PRINTFORCES\n','PRINTOPT\n','ENDOPT\n']

for i,functional in enumerate(functional_list[0]):
    gap_input = copy.deepcopy(gap_input_initial)
    
    #OPTGEOM
    #folder = './data/optgeom/%s'%functional
    folder = './data/'
    if not os.path.exists(folder):           
            os.mkdir(folder)
    # write crystal input
    for line in opt_list:
        gap_input.geom_block.insert(-1,line)
     
    for line in functional_details[i]:
        gap_input.func_block.insert(-1,line)
    
    #write the input and gui
    gap_input.write_crystal_input(os.path.join(folder,'GaP_%s_optgeom.d12'%functional))
    gap_gui_initial.write_crystal_gui(os.path.join(folder,'GaP_%s_optgeom.gui'%functional))
    print(gap_input.geom_block)
    
    #BANDS
    bands_input = Properties_input()
    bands_input.make_newk_block(12,24)    
    k_path = HighSymmKpath(gap_pmg_intial_prim)
    n_kpoints = 500
    first_band = 1
    last_band = 26
    bands_input.make_bands_block(k_path,n_kpoints,first_band,last_band)
    bands_input.write_properties_input(os.path.join(folder,'GaP_%s_bands.d3'%functional))
    
    #DOSS
    doss_input = Properties_input()
    doss_input.make_newk_block(12,24)
    doss_input.make_doss_block(n_points=500,e_range=[-5,15],plotting_option=2,poly=12,print_option=1)
    doss_input.write_properties_input(os.path.join(folder,'GaP_%s_doss.d3'%functional))
    
    #SLURM CRYSTAL
    
    slurm_file_crystal_data.append('timeout 2876m /work/e05/e05/bcamino/runCRYSTAL/Pcry_slurm GaP_%s_optgeom.d12 &\n'%functional)
    slurm_file_crystal_data.append('wait\n')
    slurm_file_crystal_data.append('/work/e05/e05/bcamino/runCRYSTAL/post_proc_slurm crys GaP_%s_optgeom.d12 &\n'%functional)
    slurm_file_crystal_data.append('wait\n\n')
    slurm_file_crystal_data.append('')

    


['EXTERNAL\n', 'OPTGEOM\n', 'FULLOPTG\n', 'FRACTION\n', 'PRINTFORCES\n', 'PRINTOPT\n', 'ENDOPT\n', 'ENDG\n']


NameError: name 'data' is not defined

## Supercell